In [1]:
import os
import io
import re
import time
import itertools
import operator
from os import listdir
from os.path import isfile, join
from io import StringIO
import pandas as pd
import numpy as np
import PyPDF2
import spacy
import textract
from difflib import SequenceMatcher
import en_core_web_sm
nlp = en_core_web_sm.load()

In [2]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [3]:
similar('india news .com',"india news x .com ")

0.9090909090909091

In [3]:
flname='C:/Users/lingadhar_duggirala/Desktop/Metadata_Validation/FDCA3Cos with AlternateNames.xlsx'
comp_master=pd.read_excel(flname)

In [4]:
comp_master=comp_master[["CompanyId","Company Name","ALTERNATENAME"]]
comp_master["compnmlower"]=comp_master["Company Name"].str.lower()
comp_master["alternatenamelower"]=comp_master["ALTERNATENAME"].str.lower()

In [5]:
comp_master=comp_master.drop(comp_master[(comp_master["compnmlower"].str.startswith('u.s. securities'))&(comp_master.compnmlower.notnull())].index)

In [7]:
comp_master["compnmlower"]=comp_master.compnmlower.fillna(value="UNK")
comp_master["Company Name"]=comp_master["Company Name"].fillna(value="UNK")

In [10]:
os.chdir('C:\\Users\\lingadhar_duggirala\\Desktop\\Metadata_Validation\\SEC Documents')

In [11]:
data=" ".join("".join([line.rstrip('\n') for line in io.open("1687129861.txt")]).split()[0:100])

In [12]:
data

'0001279569-16-00410910-Q RITCHIE BROS AUCTIONEERS INC 2016080820160808171410171410171410\x04\x01\x01 0 <SUBMISSION><ACCESSION-NUMBER>0001279569-16-004109<TYPE>10-Q<PUBLIC-DOCUMENT-COUNT>102<PERIOD>20160630<FILING-DATE>20160808<DATE-OF-FILING-DATE-CHANGE>20160808<FILER><COMPANY-DATA><CONFORMED-NAME>RITCHIE BROS AUCTIONEERS INC<CIK>0001046102<ASSIGNED-SIC>7389<IRS-NUMBER>000000000<STATE-OF-INCORPORATION>A6<FISCAL-YEAR-END>1231</COMPANY-DATA><FILING-VALUES><FORM-TYPE>10-Q<ACT>34<FILE-NUMBER>001-13425<FILM-NUMBER>161815160</FILING-VALUES><BUSINESS-ADDRESS><STREET1>9500 GLENLYON PARKWAY<CITY>BURNABY<STATE>A1<ZIP>V5J 0C6<PHONE>7783315500</BUSINESS-ADDRESS><MAIL-ADDRESS><STREET1>9500 GLENLYON PARKWAY<CITY>BURNABY<STATE>A1<ZIP>V5J 0C6</MAIL-ADDRESS></FILER><DOCUMENT><TYPE>10-Q<SEQUENCE>1Ã¯Â»Â¿FORM 10-Q UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549Ã‚Â FORM 10-QÃ‚Â x QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the quarterl

In [13]:
txt=nlp(data)
orgs=list(set([(X.text).lower() for X in txt.ents if (X.label_=='ORG')]))
orgs=[x for x in orgs if (len(x.split())>1)]
orgs

['ritchie bros auctioneers inc',
 'ritchie bros. auctioneers incorporated (exact name of registrant',
 'ã‚â commission',
 'united states securities and exchange commission']

In [39]:
orgcorpus=set(list(itertools.chain(*([i.split() for i in orgs]))))
#mystring.replace(" ", "_")
orgcorpus=[i.replace("(","") for i in orgcorpus]
orgcorpus

['inc',
 'ritchie',
 'name',
 'incorporated',
 'exact',
 'states',
 'of',
 'bros.',
 'securities',
 'registrant',
 'and',
 'auctioneers',
 'commission',
 'exchange',
 'ã‚â',
 'united',
 'bros']

In [53]:
for i in orgcorpus:
    x=comp_master[comp_master.compnmlower.str.contains(i)] 

In [74]:
start_time = time.clock()
k=0
unp_files=[]
onlyfiles = [f for f in listdir('C:\\Users\\lingadhar_duggirala\\Desktop\\Metadata_Validation\\sec_sample') if isfile(join('C:\\Users\\lingadhar_duggirala\\Desktop\\Metadata_Validation\\sec_sample', f))]
os.chdir('C:\\Users\\lingadhar_duggirala\\Desktop\\Metadata_Validation\\sec_sample')
initialdf=pd.DataFrame(columns=(['Docid','Company','companyid']))
for file in onlyfiles:
    comp=set([])
    print(file)
    data=" ".join("".join([line.rstrip('\n') for line in io.open(file)]).split()[0:100])
    #data = re.sub(r'[0-9]+', '', data)
    #data=re.sub('[^A-Za-z0-9]+', ' ', data)
    txt=nlp(data)
    orgs=list(set([(X.text).lower() for X in txt.ents if (X.label_=='ORG')]))
    orgs=[x for x in orgs if (len(x.split())>1)]
    #orgcorpus=set(list(itertools.chain(*([i.split() for i in orgs]))))
    #removetable = str.maketrans('', '', ')(+[]')
    #orgcorpus=[s.translate(removetable) for s in orgcorpus]
    for o in orgs:
        for row in set(comp_master[comp_master.compnmlower.str.startswith(o[0:3])]["compnmlower"]):
            if similar(o,row)>0.92:
                print(o,row)
                comp.add(row)
    if len(comp)>0:
        for i in comp:
            initialdf.loc[k,['Docid']]=os.path.splitext(file)[0]
            initialdf.loc[k,['Company']]=set(comp_master[comp_master.compnmlower==i]["Company Name"])
            initialdf.loc[k,['companyid']]=set(comp_master[comp_master.compnmlower==i]["CompanyId"])  
            k+=1
    else:
        data = re.sub(r'[0-9]+', '', data)
        txt=nlp(data)
        orgs=list(set([(X.text).lower() for X in txt.ents if (X.label_=='ORG')]))
        orgs=[x for x in orgs if (len(x.split())>1)]
        for o in orgs:
            for row in set(comp_master[comp_master.alternatenamelower.str.startswith(o[0:3])]["alternatenamelower"]):
                if similar(o,row)>0.92:
                    comp.add(row)
    if len(comp)>0:
        for i in comp:
            initialdf.loc[k,['Docid']]=os.path.splitext(file)[0]
            initialdf.loc[k,['Company']]=set(comp_master[comp_master.alternatenamelower==i]["Company Name"])
            initialdf.loc[k,['companyid']]=set(comp_master[comp_master.alternatenamelower==i]["CompanyId"])  
            k+=1
    else:
        initialdf.loc[k,['Docid']]=os.path.splitext(file)[0]
        initialdf.loc[k,['Company']]="XXNo LuckXX"
        initialdf.loc[k,['companyid']]="XX0000XX"
        k+=1

print('Time took for preprocessing the text :',(time.clock() - start_time)/60, "minutes")

initialdf['Company'] = initialdf['Company'].map(lambda x: str(x)[:-2])
initialdf['Company'] = initialdf['Company'].map(lambda x: str(x)[2:])
initialdf['companyid'] = initialdf['companyid'].map(lambda x: str(x)[1:])
initialdf['companyid'] = initialdf['companyid'].map(lambda x: str(x)[:-1])
initialdf.drop_duplicates(keep="first", inplace=True)
initialdf

C:\Users\lingadhar_duggirala\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


230288860.txt
230292662.txt
mid america apartment communities inc mid-america apartment communities, inc.
242236708.txt
cleartronic, inc. cleartronic, inc.
243576070.txt
253874620.txt
kimco realty corporation kimco realty corporation
258140358.txt
258370180.txt
east west bancorp inc east west bancorp, inc.
259118202.txt
259720618.txt
key energy services inc key energy services, inc.
261066672.txt
akamai technologies inc akamai technologies, inc.
261273360.txt
261655748.txt
nbt bancorp inc nbt bancorp inc.
267882938.txt
wheeler real estate investment trust wheeler real estate investment trust, inc.
wheeler real estate investment trust, inc. wheeler real estate investment trust, inc.
270663546.txt
northwest natural holding company northwest natural holding company
273075380.txt
273186958.txt
275798274.txt
arena pharmaceuticals inc arena pharmaceuticals, inc.
278984340.txt
manning & napier, inc. manning & napier, inc.
279035452.txt
kingsway financial services inc kingsway financial servic

C:\Users\lingadhar_duggirala\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:51: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


Time took for preprocessing the text : 0.45167491403359844 minutes


,Docid,Company,companyid
0,230288860,No Luck,X0000X
1,230292662,"Mid-America Apartment Communities, Inc.",332358
2,230292662,t,et(
3,242236708,"Cleartronic, Inc.",27874526
4,242236708,t,et(
5,243576070,No Luck,X0000X
6,253874620,Kimco Realty Corporation,320371
7,253874620,t,et(
8,258140358,No Luck,X0000X
9,258370180,"East West Bancorp, Inc.",396680


In [63]:
start_time = time.clock()
k=0
onlyfiles = [f for f in listdir('C:\\Users\\lingadhar_duggirala\\Desktop\\Metadata_Validation\\sec_sample') if isfile(join('C:\\Users\\lingadhar_duggirala\\Desktop\\Metadata_Validation\\sec_sample', f))]
os.chdir('C:\\Users\\lingadhar_duggirala\\Desktop\\Metadata_Validation\\sec_sample')
initialdf=pd.DataFrame(columns=(['Docid','Company','companyid']))
for file in onlyfiles:
    comp=dict()
    print(file)
    data=" ".join("".join([line.rstrip('\n') for line in io.open(file)]).split()[0:100])
    #data = re.sub(r'[0-9]+', '', data)
    #data=re.sub('[^A-Za-z0-9]+', ' ', data)
    txt=nlp(data)
    orgs=list(set([(X.text).lower() for X in txt.ents if (X.label_=='ORG')]))
    orgs=[x for x in orgs if (len(x.split())>1)]
    orgcorpus=set(list(itertools.chain(*([i.split() for i in orgs]))))
    orgcorpus=[i.replace("(","") for i in orgcorpus]
    orgcorpus=[i.replace(")","") for i in orgcorpus]
    for i in orgcorpus:
        x=comp_master[comp_master.compnmlower.str.contains(i)] 
    for o in orgs:
        for row in x["compnmlower"]:
            if similar(o,row)>0.85:
                comp[row]=similar(o,row)
        if len(comp)>0:
            y=max(comp, key=comp.get)
            initialdf.loc[k,['Docid']]=os.path.splitext(file)[0]
            initialdf.loc[k,['Company']]=set(comp_master[comp_master.compnmlower==y]["Company Name"])
            initialdf.loc[k,['companyid']]=set(comp_master[comp_master.compnmlower==y]["CompanyId"])  
            k+=1
        else:
            continue
print('Time took for preprocessing the text :',(time.clock() - start_time)/60, "minutes")

initialdf['Company'] = initialdf['Company'].map(lambda x: str(x)[:-2])
initialdf['Company'] = initialdf['Company'].map(lambda x: str(x)[2:])
initialdf['companyid'] = initialdf['companyid'].map(lambda x: str(x)[1:])
initialdf['companyid'] = initialdf['companyid'].map(lambda x: str(x)[:-1])
initialdf.drop_duplicates(keep="first", inplace=True)
initialdf

C:\Users\lingadhar_duggirala\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


230288860.txt
230292662.txt
242236708.txt
243576070.txt
253874620.txt
258140358.txt
258370180.txt
259118202.txt
259720618.txt
261066672.txt
261273360.txt
261655748.txt
267882938.txt
270663546.txt
273075380.txt
273186958.txt
275798274.txt
278984340.txt
279035452.txt
285661030.txt
491648246.txt


C:\Users\lingadhar_duggirala\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


Time took for preprocessing the text : 1.048295178883321 minutes


,Docid,Company,companyid
0,230292662,"Mid-America Apartment Communities, Inc.",332358
2,259720618,Aarow Environmental Group Inc,4019629
7,261066672,"Akamai Technologies, Inc.",24522
9,267882938,CT Real Estate Investment Trust,245298245
10,267882938,"Wheeler Real Estate Investment Trust, Inc.",139437757
15,273075380,"KBS Strategic Opportunity REIT, Inc.",52056017
17,278984340,"Manning & Napier, Inc.",136932229
20,279035452,Kingsway Financial Services Inc.,879946


In [57]:
initialdf

,Docid,Company,companyid
0,230288860,No Luck,No Luck
1,230288860,No Luck,No Luck
2,230288860,No Luck,No Luck


In [73]:
x="hi hey oye".split()

In [ ]:
start_time = time.clock()
k=0
unp_files=[]
onlyfiles = [f for f in listdir('C:\\Users\\lingadhar_duggirala\\Desktop\\Metadata_Validation\\sec_sample') if isfile(join('C:\\Users\\lingadhar_duggirala\\Desktop\\Metadata_Validation\\sec_sample', f))]
os.chdir('C:\\Users\\lingadhar_duggirala\\Desktop\\Metadata_Validation\\sec_sample')
initialdf=pd.DataFrame(columns=(['Docid','Company','companyid']))
for file in onlyfiles:
    comp=set([])
    print(file)
    data=" ".join("".join([line.rstrip('\n') for line in io.open(file)]).split()[0:100])
    #data = re.sub(r'[0-9]+', '', data)
    #data=re.sub('[^A-Za-z0-9]+', ' ', data)
    txt=nlp(data)
    orgs=list(set([(X.text).lower() for X in txt.ents if (X.label_=='ORG')]))
    orgs=[x for x in orgs if (len(x.split())>1)]
    #orgcorpus=set(list(itertools.chain(*([i.split() for i in orgs]))))
    #removetable = str.maketrans('', '', ')(+[]')
    #orgcorpus=[s.translate(removetable) for s in orgcorpus]
    for org in orgs:
        orgcorpus=o.split()
        for o in orgcorpus:
            for row in set(comp_master[comp_master.compnmlower.str.contains(o)]["compnmlower"]):
                if similar(o,row)>0.92:
                    print(o,row)
                    comp.add(row)
    if len(comp)>0:
        for i in comp:
            initialdf.loc[k,['Docid']]=os.path.splitext(file)[0]
            initialdf.loc[k,['Company']]=set(comp_master[comp_master.compnmlower==i]["Company Name"])
            initialdf.loc[k,['companyid']]=set(comp_master[comp_master.compnmlower==i]["CompanyId"])  
            k+=1
    else:
        data = re.sub(r'[0-9]+', '', data)
        txt=nlp(data)
        orgs=list(set([(X.text).lower() for X in txt.ents if (X.label_=='ORG')]))
        orgs=[x for x in orgs if (len(x.split())>1)]
        for o in orgs:
            for row in set(comp_master[comp_master.alternatenamelower.str.startswith(o[0:3])]["alternatenamelower"]):
                if similar(o,row)>0.92:
                    comp.add(row)
    if len(comp)>0:
        for i in comp:
            initialdf.loc[k,['Docid']]=os.path.splitext(file)[0]
            initialdf.loc[k,['Company']]=set(comp_master[comp_master.alternatenamelower==i]["Company Name"])
            initialdf.loc[k,['companyid']]=set(comp_master[comp_master.alternatenamelower==i]["CompanyId"])  
            k+=1
    else:
        initialdf.loc[k,['Docid']]=os.path.splitext(file)[0]
        initialdf.loc[k,['Company']]="XXNo LuckXX"
        initialdf.loc[k,['companyid']]="XX0000XX"
        k+=1

print('Time took for preprocessing the text :',(time.clock() - start_time)/60, "minutes")

initialdf['Company'] = initialdf['Company'].map(lambda x: str(x)[:-2])
initialdf['Company'] = initialdf['Company'].map(lambda x: str(x)[2:])
initialdf['companyid'] = initialdf['companyid'].map(lambda x: str(x)[1:])
initialdf['companyid'] = initialdf['companyid'].map(lambda x: str(x)[:-1])
initialdf.drop_duplicates(keep="first", inplace=True)
initialdf